In [86]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_c0.dta"
)
print(C.shape)
C.sample(5)

(91319, 21)


,hhcode,sec,msno,scq01,scq02,scq03,scqo4,scq05,scq06,scq07,...,scq08b,scq09a,scq09b,psu,quarter,region,psu_hies,hhno,hhcode_hies,province
75735,3.231001e+09,C,10,NaN,NaN,yes,class 1,yes,class 2,1 government,...,NaN,NaN,NaN,32310010,4,urban,30710401,6,3.071040e+09,nwfp
26091,1.232002e+09,C,2,no,no,no,NaN,NaN,NaN,NaN,...,NaN,12 lack of male/female staff,NaN,12320019,3,rural,12320302,2,1.232030e+09,punjab
88284,4.192001e+09,C,6,NaN,NaN,yes,class 2,yes,class 3,1 government,...,3 lack of books,NaN,NaN,41920013,4,rural,40520402,14,4.052040e+09,balochistan
77087,3.241001e+09,C,6,NaN,NaN,yes,0: less than class1,yes,0: less than class1,3 private,...,NaN,NaN,NaN,32410006,1,urban,30710103,7,3.071010e+09,nwfp
29844,1.272001e+09,C,3,yes,yes,yes,class 6,no,NaN,NaN,...,NaN,3 too expansive,10 employment,12720005,4,rural,12720401,16,1.272040e+09,punjab


In [87]:
C.columns

Index(['hhcode', 'sec', 'msno', 'scq01', 'scq02', 'scq03', 'scqo4', 'scq05',
       'scq06', 'scq07', 'scq08a', 'scq08b', 'scq09a', 'scq09b', 'psu',
       'quarter', 'region', 'psu_hies', 'hhno', 'hhcode_hies', 'province'],
      dtype='object')

In [88]:
# Reduce to the questions of interest (4 questions):

C["ever_admitted"] = C["scq03"]
C["max_level_acheived"] = C["scqo4"]
C["currently_enrrolled"] = C["scq05"] # IMPORTANT!!! (counts of enrrollment)
C["why_not"] = C["scq09a"]

C_relevant = C[
    [
        "hhcode",
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province",
        "psu_hies",
        "hhno",
        "hhcode_hies",
    ]
].copy()


In [89]:
C_relevant

,hhcode,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province,psu_hies,hhno,hhcode_hies
0,1.001100e+09,yes,class 10,no,2 education completed,urban,punjab,11711101,1,1.171110e+09
1,1.001100e+09,yes,cl;ass 8,no,2 education completed,urban,punjab,11711101,1,1.171110e+09
2,1.001100e+09,yes,17: ma/msc,no,2 education completed,urban,punjab,11711101,1,1.171110e+09
3,1.001100e+09,yes,class 10,yes,NaN,urban,punjab,11711101,1,1.171110e+09
4,1.001100e+09,yes,12: ba/bsc,no,10 employment,urban,punjab,11711101,2,1.171110e+09
...,...,...,...,...,...,...,...,...,...,...
91314,4.262002e+09,no,NaN,NaN,15 other,rural,balochistan,40620404,15,4.062040e+09
91315,4.262002e+09,no,NaN,NaN,10 employment,rural,balochistan,40620404,16,4.062040e+09
91316,4.262002e+09,no,NaN,NaN,13 parent did not approve,rural,balochistan,40620404,16,4.062040e+09
91317,4.262002e+09,no,NaN,NaN,13 parent did not approve,rural,balochistan,40620404,16,4.062040e+09


In [90]:
B = pd.read_stata(
    "/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/00_source_data/0405_sec_b0.dta"
)
B.sample(5)

# Change the names of the questions for readability
B['sex'] = B['sbq01']
B['age'] = B['sbq04']
B['marital_status'] = B['sbq05']

B_relevant = B[['hhcode', 'age', 'sex', 'marital_status']].copy()

In [91]:
B.sample(5)

,hhcode,sec,msno,sbq01,sbq02,sbq03,sbq04,sbq05,weight,psu,quarter,region,psu_hies,hhno,hhcode_hies,province,sex,age,marital_status
21680,1.181209e+09,B,5,male,present,0:non relative,19,1:never married,2676.836914,11812094,3,urban,10912304,5,1.091230e+09,punjab,male,19,1:never married
85075,3.232001e+09,B,3,female,present,3:son/daughter,8,1:never married,1639.745972,32320006,3,rural,32320301,2,3.232030e+09,nwfp,female,8,1:never married
60124,2.161107e+09,B,5,male,present,3:son/daughter,6,1:never married,1717.208984,21611067,4,urban,20511404,12,2.051140e+09,sindh,male,6,1:never married
53200,2.102003e+09,B,8,male,present,3:son/daughter,9,1:never married,2804.279053,21020025,3,rural,21020301,11,2.102030e+09,sindh,male,9,1:never married
83838,3.222001e+09,B,15,female,present,4:grand son / grand daughter,3,1:never married,1111.203979,32220008,2,rural,32220201,2,3.222020e+09,nwfp,female,3,1:never married


In [92]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [93]:
# MERGE
years04_05 = pd.merge(B_relevant, C_relevant, on=["hhcode"], how='inner', indicator=False)

# Evaluate merge 
#years04_05._merge.value_counts()

In [94]:
# Select people in school enrollment age (I would say 4-15 years old)

years04_05 = years04_05.loc[(years04_05["age"] > 4)&(years04_05["age"] <= 15)]
years04_05['year']=2004

In [95]:
# Change dtype to string (hhcode)
years04_05["hhcode"] = years04_05["hhcode"].astype("str")

In [96]:
#years04_05['subprovince']=years04_05['hhcode'].str.extract(r'(^w{3})')
years04_05['subprovince']= years04_05['hhcode'].apply(lambda x: x[0:3])

In [97]:
years04_05["subprovince"] = years04_05["subprovince"].astype("int")

In [98]:
years04_05.loc[(years04_05["subprovince"] > 116) & (years04_05["subprovince"] < 118)]

,hhcode,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province,psu_hies,hhno,hhcode_hies,year,subprovince
134233,1171000302.0,13,female,1:never married,yes,class 10,no,14 child not willing,urban,punjab,10410304,2,1.041030e+09,2004,117
134234,1171000302.0,13,female,1:never married,yes,cl;ass 8,no,6 had to help with work,urban,punjab,10410304,2,1.041030e+09,2004,117
134235,1171000302.0,13,female,1:never married,yes,class 10,no,3 too expansive,urban,punjab,10410304,2,1.041030e+09,2004,117
134236,1171000302.0,13,female,1:never married,yes,cl;ass 8,no,3 too expansive,urban,punjab,10410304,2,1.041030e+09,2004,117
134237,1171000302.0,13,female,1:never married,yes,cl;ass 8,no,3 too expansive,urban,punjab,10410304,2,1.041030e+09,2004,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137819,1172002316.0,6,female,1:never married,yes,class 5,no,3 too expansive,rural,punjab,11720301,16,1.172030e+09,2004,117
137820,1172002316.0,6,female,1:never married,yes,class 4,yes,NaN,rural,punjab,11720301,16,1.172030e+09,2004,117
137821,1172002316.0,6,female,1:never married,yes,class 1,yes,NaN,rural,punjab,11720301,16,1.172030e+09,2004,117
137822,1172002316.0,6,female,1:never married,yes,0: less than class1,yes,NaN,rural,punjab,11720301,16,1.172030e+09,2004,117


In [99]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0405_v1.csv"
)
strat_name.sample(5)

,province,subprovince,Stratum Name
85,Balochistan,412,412
72,NWFP,322,LakkiMarwat
73,NWFP,323,Mardan
77,Balochistan,403,Kalat
13,Punjab,113,Gujrat


In [102]:
# MERGE
years04_05_ = pd.merge(years04_05, strat_name, on=["subprovince"], how='inner', indicator=True)

# Evaluate merge 
years04_05_._merge.value_counts()

both          262327
left_only          0
right_only         0
Name: _merge, dtype: int64